In [23]:
!pip install pandas dash

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels


In [24]:
!pip install jupyter-dash


Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels


In [27]:
from dash import Dash, dcc, html, Input, Output
import pandas as pd
import plotly.express as px

df0 = pd.read_csv('spacex_launch_dash.csv')
data = pd.DataFrame(df0)


app = Dash(__name__)

# Step 1: Define Layout
app.layout = html.Div(children=[
                html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                dcc.Dropdown(id='site-dropdown',
                options=[
                    {'label': 'All Sites', 'value': 'ALL'},
                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                ],
                value='ALL',
                placeholder="Choose a Site",
                searchable=True
                ),
                html.Br(),

                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                html.Div(dcc.Graph(id='success-pie-chart')),
                html.Br(),

                html.P("Payload range (Kg):"),
                # TASK 3: Add a slider to select payload range
                
                dcc.RangeSlider(id='payload-slider',
                min=0, max=10000, step=1000,
                marks={i: str(i) for i in range(0, 100001, 1000)},
                value=[0, 5000]
                ),

                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                html.Div(dcc.Graph(id='success-payload-scatter-chart')),

                ])
               

# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
             

def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        data_all = data[data['class'] == 1]
        fig = px.pie(data_all, names='Launch Site', 
        title='Success for All Sites')
    else:
        data_site = data[data['Launch Site'] == entered_site]
        fig = px.pie(data_site, names='class',  
        title = f'Success/Fail for {entered_site}')
    return fig


@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'),
              Input(component_id="payload-slider", component_property="value"))

def get_scatter_chart(entered_site, pload):
    lower, upper = pload
    data_load = data[(data['Payload Mass (kg)'] >= lower) & (data['Payload Mass (kg)'] <= upper)]
    data_load['BV'] = data_load['Booster Version'].str.split(" ").str[1]
    
    if entered_site == 'ALL':
        fig2 = px.scatter(data_load, x='Payload Mass (kg)', y='class', color='BV', 
        title='Correlation between Payload and Success for All Sites')
    else:
        data_load_site = data_load[data_load['Launch Site'] == entered_site]
        fig2 = px.scatter(data_load_site, x='Payload Mass (kg)', y='class', color='BV', 
        title= f"Correlation between Payload and Success for {entered_site}")
    return fig2
    
# Step 3: Run the App
if __name__ == "__main__":
    app.run_server(debug=True)





In [65]:
data.columns


Index(['Unnamed: 0', 'Flight Number', 'Launch Site', 'class',
       'Payload Mass (kg)', 'Booster Version', 'Booster Version Category'],
      dtype='object')